# Named Entity Recognition (NER) and N-gram Language Models

This notebook covers:
1. **Named Entity Recognition (NER)**
   - Dictionary-based methods
   - CRF-based methods
2. **N-gram Language Models**
   - Smoothing techniques
   - Backoff methods
   - Perplexity evaluation

In [ ]:
# Install required packages
# !pip install sklearn-crfsuite nltk spacy
# !python -m spacy download en_core_web_sm

In [ ]:
import re
import numpy as np
import pandas as pd
from collections import defaultdict, Counter
import warnings
warnings.filterwarnings('ignore')

---
## Part 1: Named Entity Recognition (NER)

Named Entity Recognition is the task of identifying and classifying named entities in text into predefined categories such as:
- **PERSON**: Names of people
- **ORGANIZATION**: Companies, agencies, institutions
- **LOCATION**: Countries, cities, states
- **DATE**: Absolute or relative dates
- **MONEY**: Monetary values
- etc.

### 1.1 Dictionary-based NER

Dictionary-based NER uses predefined lists of entities to identify and classify named entities in text.

**Advantages:**
- Simple to implement
- Fast and efficient
- High precision for known entities

**Disadvantages:**
- Limited to known entities
- Cannot handle variations or new entities
- Requires maintenance of dictionaries

In [ ]:
class DictionaryBasedNER:
    """Simple dictionary-based Named Entity Recognition"""
    
    def __init__(self):
        # Initialize entity dictionaries
        self.entity_dicts = {
            'PERSON': {
                'John Smith', 'Mary Johnson', 'Robert Brown', 'Emma Watson',
                'Bill Gates', 'Steve Jobs', 'Elon Musk', 'Jeff Bezos'
            },
            'ORGANIZATION': {
                'Google', 'Microsoft', 'Apple', 'Amazon', 'Tesla',
                'Stanford University', 'MIT', 'NASA', 'FBI', 'United Nations'
            },
            'LOCATION': {
                'New York', 'London', 'Paris', 'Tokyo', 'California',
                'United States', 'England', 'France', 'Japan', 'India'
            },
            'DATE': {
                'Monday', 'Tuesday', 'January', 'February', 'today',
                'yesterday', 'tomorrow', 'next week'
            }
        }
        
        # Create a combined lookup dictionary for faster matching
        self.entity_lookup = {}
        for entity_type, entities in self.entity_dicts.items():
            for entity in entities:
                self.entity_lookup[entity.lower()] = entity_type
    
    def add_entity(self, entity_type, entity_name):
        """Add a new entity to the dictionary"""
        if entity_type not in self.entity_dicts:
            self.entity_dicts[entity_type] = set()
        self.entity_dicts[entity_type].add(entity_name)
        self.entity_lookup[entity_name.lower()] = entity_type
    
    def recognize(self, text):
        """Recognize entities in text using dictionary matching"""
        entities = []
        words = text.split()
        
        i = 0
        while i < len(words):
            # Try matching multi-word entities (up to 3 words)
            for n in range(min(3, len(words) - i), 0, -1):
                phrase = ' '.join(words[i:i+n])
                phrase_clean = re.sub(r'[^\w\s]', '', phrase)  # Remove punctuation
                
                if phrase_clean.lower() in self.entity_lookup:
                    entity_type = self.entity_lookup[phrase_clean.lower()]
                    entities.append({
                        'text': phrase_clean,
                        'type': entity_type,
                        'start': i,
                        'end': i + n
                    })
                    i += n
                    break
            else:
                i += 1
        
        return entities
    
    def display_entities(self, text):
        """Display recognized entities with their types"""
        entities = self.recognize(text)
        print(f"Text: {text}\n")
        print("Recognized Entities:")
        for entity in entities:
            print(f"  - {entity['text']:20s} => {entity['type']}")
        return entities

In [ ]:
# Example usage of Dictionary-based NER
dict_ner = DictionaryBasedNER()

sample_texts = [
    "Bill Gates founded Microsoft in the United States.",
    "Elon Musk runs Tesla and lives in California.",
    "Emma Watson studied at Stanford University in California."
]

for text in sample_texts:
    dict_ner.display_entities(text)
    print("-" * 60)

### 1.2 CRF-based NER

Conditional Random Fields (CRF) is a probabilistic model for sequence labeling that considers:
- **Context**: Surrounding words and their features
- **Sequential dependencies**: Relationships between adjacent labels
- **Features**: Word shape, capitalization, POS tags, etc.

**Advantages:**
- Can learn from training data
- Handles unknown entities
- Considers context and sequence

**Disadvantages:**
- Requires labeled training data
- More complex than dictionary-based
- Slower inference

In [ ]:
# Feature extraction for CRF
def word2features(sent, i):
    """Extract features for a word at position i in sentence"""
    word = sent[i][0]
    
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
    }
    
    # Features for previous word
    if i > 0:
        word1 = sent[i-1][0]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
        })
    else:
        features['BOS'] = True  # Beginning of sentence
    
    # Features for next word
    if i < len(sent) - 1:
        word1 = sent[i+1][0]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
        })
    else:
        features['EOS'] = True  # End of sentence
    
    return features

def sent2features(sent):
    """Extract features for all words in a sentence"""
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    """Extract labels for all words in a sentence"""
    return [label for token, label in sent]

def sent2tokens(sent):
    """Extract tokens from a sentence"""
    return [token for token, label in sent]

In [ ]:
# Sample training data in BIO format
# B-X: Beginning of entity type X
# I-X: Inside (continuation) of entity type X
# O: Outside (not an entity)

train_sentences = [
    [('Bill', 'B-PERSON'), ('Gates', 'I-PERSON'), ('founded', 'O'), 
     ('Microsoft', 'B-ORG'), ('in', 'O'), ('1975', 'B-DATE')],
    
    [('Steve', 'B-PERSON'), ('Jobs', 'I-PERSON'), ('co-founded', 'O'), 
     ('Apple', 'B-ORG'), ('in', 'O'), ('California', 'B-LOC')],
    
    [('Elon', 'B-PERSON'), ('Musk', 'I-PERSON'), ('leads', 'O'), 
     ('Tesla', 'B-ORG'), ('and', 'O'), ('SpaceX', 'B-ORG')],
    
    [('Google', 'B-ORG'), ('was', 'O'), ('founded', 'O'), ('in', 'O'), 
     ('California', 'B-LOC'), ('by', 'O'), ('Larry', 'B-PERSON'), ('Page', 'I-PERSON')],
    
    [('Amazon', 'B-ORG'), ('is', 'O'), ('based', 'O'), ('in', 'O'), 
     ('Seattle', 'B-LOC'), (',', 'O'), ('Washington', 'B-LOC')],
]

test_sentences = [
    [('Jeff', 'B-PERSON'), ('Bezos', 'I-PERSON'), ('founded', 'O'), 
     ('Amazon', 'B-ORG'), ('in', 'O'), ('1994', 'B-DATE')],
    
    [('Mark', 'B-PERSON'), ('Zuckerberg', 'I-PERSON'), ('created', 'O'), 
     ('Facebook', 'B-ORG'), ('at', 'O'), ('Harvard', 'B-ORG')],
]

In [ ]:
# Train CRF model
try:
    import sklearn_crfsuite
    from sklearn_crfsuite import metrics
    
    # Prepare training data
    X_train = [sent2features(s) for s in train_sentences]
    y_train = [sent2labels(s) for s in train_sentences]
    
    X_test = [sent2features(s) for s in test_sentences]
    y_test = [sent2labels(s) for s in test_sentences]
    
    # Train CRF model
    crf = sklearn_crfsuite.CRF(
        algorithm='lbfgs',
        c1=0.1,
        c2=0.1,
        max_iterations=100,
        all_possible_transitions=True
    )
    
    crf.fit(X_train, y_train)
    
    # Make predictions
    y_pred = crf.predict(X_test)
    
    # Display results
    print("CRF-based NER Results:\n")
    for i, sent in enumerate(test_sentences):
        print(f"Sentence {i+1}:")
        for j, (token, true_label) in enumerate(sent):
            pred_label = y_pred[i][j]
            match = "✓" if true_label == pred_label else "✗"
            print(f"  {token:15s} True: {true_label:10s} Pred: {pred_label:10s} {match}")
        print()
    
    # Calculate metrics
    labels = list(crf.classes_)
    labels.remove('O')  # Remove 'O' label for evaluation
    
    print("\nModel Performance:")
    print(metrics.flat_classification_report(
        y_test, y_pred, labels=labels, digits=3
    ))
    
except ImportError:
    print("sklearn-crfsuite not installed. Run: pip install sklearn-crfsuite")

### 1.3 Comparison: Dictionary-based vs CRF-based NER

| Aspect | Dictionary-based | CRF-based |
|--------|-----------------|----------|
| **Training Data** | Not required | Required |
| **New Entities** | Cannot detect | Can detect |
| **Context Awareness** | No | Yes |
| **Speed** | Very fast | Slower |
| **Precision** | High (for known) | Moderate to High |
| **Recall** | Low | Moderate to High |
| **Maintenance** | Manual updates | Automatic learning |

---
## Part 2: N-gram Language Models

N-gram language models predict the probability of a word given the previous (n-1) words.

**Types:**
- **Unigram (1-gram)**: P(w) - probability of a word
- **Bigram (2-gram)**: P(w|w₁) - probability given previous word
- **Trigram (3-gram)**: P(w|w₁,w₂) - probability given previous 2 words

**Formula (Bigram):**
```
P(w₂|w₁) = Count(w₁, w₂) / Count(w₁)
```

In [ ]:
class NgramLanguageModel:
    """N-gram language model with smoothing and backoff"""
    
    def __init__(self, n=2):
        self.n = n
        self.ngram_counts = defaultdict(int)
        self.context_counts = defaultdict(int)
        self.vocabulary = set()
        self.total_words = 0
    
    def tokenize(self, text):
        """Simple tokenization"""
        # Add sentence boundaries
        tokens = ['<s>'] * (self.n - 1) + text.lower().split() + ['</s>']
        return tokens
    
    def train(self, corpus):
        """Train the n-gram model on a corpus"""
        for text in corpus:
            tokens = self.tokenize(text)
            self.vocabulary.update(tokens)
            self.total_words += len(tokens)
            
            # Count n-grams
            for i in range(len(tokens) - self.n + 1):
                ngram = tuple(tokens[i:i + self.n])
                context = ngram[:-1]
                
                self.ngram_counts[ngram] += 1
                self.context_counts[context] += 1
    
    def probability(self, ngram, smoothing='none'):
        """Calculate probability of an n-gram with optional smoothing"""
        if isinstance(ngram, str):
            ngram = tuple(ngram.lower().split())
        
        if smoothing == 'none':
            return self._prob_no_smoothing(ngram)
        elif smoothing == 'laplace':
            return self._prob_laplace(ngram)
        elif smoothing == 'add_k':
            return self._prob_add_k(ngram, k=0.5)
        else:
            raise ValueError(f"Unknown smoothing method: {smoothing}")
    
    def _prob_no_smoothing(self, ngram):
        """Probability without smoothing (MLE)"""
        context = ngram[:-1]
        context_count = self.context_counts[context]
        
        if context_count == 0:
            return 0.0
        
        return self.ngram_counts[ngram] / context_count
    
    def _prob_laplace(self, ngram):
        """Laplace (add-one) smoothing"""
        context = ngram[:-1]
        V = len(self.vocabulary)
        
        numerator = self.ngram_counts[ngram] + 1
        denominator = self.context_counts[context] + V
        
        return numerator / denominator
    
    def _prob_add_k(self, ngram, k=0.5):
        """Add-k smoothing"""
        context = ngram[:-1]
        V = len(self.vocabulary)
        
        numerator = self.ngram_counts[ngram] + k
        denominator = self.context_counts[context] + k * V
        
        return numerator / denominator
    
    def perplexity(self, test_corpus, smoothing='laplace'):
        """Calculate perplexity on test corpus"""
        total_log_prob = 0
        total_words = 0
        
        for text in test_corpus:
            tokens = self.tokenize(text)
            
            for i in range(len(tokens) - self.n + 1):
                ngram = tuple(tokens[i:i + self.n])
                prob = self.probability(ngram, smoothing=smoothing)
                
                if prob > 0:
                    total_log_prob += np.log2(prob)
                else:
                    # Assign small probability for unseen n-grams
                    total_log_prob += np.log2(1e-10)
                
                total_words += 1
        
        # Perplexity = 2^(-log_prob / N)
        avg_log_prob = total_log_prob / total_words
        perplexity = 2 ** (-avg_log_prob)
        
        return perplexity

### 2.1 Training an N-gram Model

In [ ]:
# Sample training corpus
train_corpus = [
    "the cat sat on the mat",
    "the dog sat on the log",
    "the cat sat on the log",
    "the dog sat on the mat",
    "a cat is a pet",
    "a dog is a pet",
    "the cat likes the mat",
    "the dog likes the log",
]

# Train bigram model
bigram_model = NgramLanguageModel(n=2)
bigram_model.train(train_corpus)

print(f"Vocabulary size: {len(bigram_model.vocabulary)}")
print(f"Total words: {bigram_model.total_words}")
print(f"Number of unique bigrams: {len(bigram_model.ngram_counts)}")

### 2.2 Smoothing Techniques

Smoothing addresses the **zero probability problem** where unseen n-grams get probability 0.

#### Types of Smoothing:

1. **Laplace (Add-One) Smoothing**
   - Add 1 to all counts
   - Formula: P(w|context) = (Count + 1) / (Context_Count + V)

2. **Add-k Smoothing**
   - Add k (0 < k < 1) to all counts
   - Formula: P(w|context) = (Count + k) / (Context_Count + k×V)

3. **Good-Turing Smoothing**
   - Use frequency of frequencies
   - Redistributes probability mass from seen to unseen n-grams

In [ ]:
# Compare probabilities with different smoothing methods
test_bigrams = [
    ('the', 'cat'),    # Seen bigram
    ('the', 'dog'),    # Seen bigram
    ('the', 'bird'),   # Unseen bigram
    ('cat', 'sat'),    # Seen bigram
]

print("Bigram Probabilities with Different Smoothing:\n")
print(f"{'Bigram':<20} {'No Smoothing':<15} {'Laplace':<15} {'Add-k (k=0.5)':<15}")
print("-" * 70)

for bigram in test_bigrams:
    prob_none = bigram_model.probability(bigram, smoothing='none')
    prob_laplace = bigram_model.probability(bigram, smoothing='laplace')
    prob_add_k = bigram_model.probability(bigram, smoothing='add_k')
    
    bigram_str = f"{bigram[0]} {bigram[1]}"
    print(f"{bigram_str:<20} {prob_none:<15.6f} {prob_laplace:<15.6f} {prob_add_k:<15.6f}")

### 2.3 Backoff Methods

Backoff uses lower-order n-grams when higher-order n-grams are not available.

**Katz Backoff:**
```
P(wₙ|w₁...wₙ₋₁) = {
    P*(wₙ|w₁...wₙ₋₁)           if count(w₁...wₙ) > 0
    α(w₁...wₙ₋₁) × P(wₙ|w₂...wₙ₋₁)   otherwise
}
```

where α is a backoff weight to ensure probabilities sum to 1.

In [ ]:
class BackoffLanguageModel:
    """N-gram language model with backoff"""
    
    def __init__(self, max_n=3):
        self.max_n = max_n
        # Train models of different orders
        self.models = {n: NgramLanguageModel(n=n) for n in range(1, max_n + 1)}
    
    def train(self, corpus):
        """Train all n-gram models"""
        for n, model in self.models.items():
            model.train(corpus)
    
    def probability_with_backoff(self, ngram, smoothing='laplace'):
        """Calculate probability using backoff"""
        if isinstance(ngram, str):
            ngram = tuple(ngram.lower().split())
        
        n = len(ngram)
        
        # Try from highest order to lowest
        for order in range(min(n, self.max_n), 0, -1):
            current_ngram = ngram[-order:]
            model = self.models[order]
            
            prob = model.probability(current_ngram, smoothing=smoothing)
            
            if prob > 0 or order == 1:
                return prob, order
        
        return 0.0, 0

In [ ]:
# Train backoff model
backoff_model = BackoffLanguageModel(max_n=3)
backoff_model.train(train_corpus)

# Test backoff behavior
test_trigrams = [
    ('the', 'cat', 'sat'),     # Seen trigram
    ('the', 'cat', 'jumped'),  # Unseen trigram, backs off to bigram
    ('the', 'bird', 'flew'),   # Unseen, backs off further
]

print("Backoff Behavior:\n")
for trigram in test_trigrams:
    prob, order_used = backoff_model.probability_with_backoff(trigram)
    trigram_str = ' '.join(trigram)
    print(f"Trigram: {trigram_str:<25} Prob: {prob:.6f}  (used {order}-gram)")

### 2.4 Perplexity

Perplexity measures how well a language model predicts a test corpus.

**Formula:**
```
Perplexity = 2^(-1/N × Σ log₂ P(wᵢ|context))
```

**Interpretation:**
- Lower perplexity = better model
- Perplexity of K means the model is as confused as if it had to choose uniformly among K possibilities
- Typical values: 50-1000 for natural language

In [ ]:
# Test corpus (similar but different from training)
test_corpus = [
    "the cat sat on the mat",
    "the dog sat on the log",
    "a cat is a good pet",
]

# Calculate perplexity with different smoothing
print("Perplexity Comparison:\n")

for smoothing in ['laplace', 'add_k']:
    perplexity = bigram_model.perplexity(test_corpus, smoothing=smoothing)
    print(f"{smoothing.capitalize():15s} smoothing: {perplexity:.2f}")

In [ ]:
# Compare different n-gram orders
print("\nPerplexity by N-gram Order (with Laplace smoothing):\n")

for n in range(1, 4):
    model = NgramLanguageModel(n=n)
    model.train(train_corpus)
    perplexity = model.perplexity(test_corpus, smoothing='laplace')
    print(f"{n}-gram model: {perplexity:.2f}")

### 2.5 Practical Example: Text Generation

In [ ]:
def generate_text(model, start_token='<s>', max_length=10, smoothing='laplace'):
    """Generate text using n-gram model"""
    generated = [start_token] * (model.n - 1)
    
    for _ in range(max_length):
        # Get context
        context = tuple(generated[-(model.n - 1):])
        
        # Calculate probabilities for all possible next words
        candidates = {}
        for word in model.vocabulary:
            if word not in ['<s>', '</s>']:
                ngram = context + (word,)
                prob = model.probability(ngram, smoothing=smoothing)
                candidates[word] = prob
        
        # Normalize probabilities
        total = sum(candidates.values())
        if total > 0:
            candidates = {w: p/total for w, p in candidates.items()}
        
        # Sample next word
        if candidates:
            words = list(candidates.keys())
            probs = list(candidates.values())
            next_word = np.random.choice(words, p=probs)
            generated.append(next_word)
            
            if next_word == '</s>':
                break
        else:
            break
    
    # Remove start tokens and return
    return ' '.join([w for w in generated if w not in ['<s>', '</s>']])

In [ ]:
# Generate some text samples
print("Generated Text Samples (Bigram Model):\n")
for i in range(5):
    text = generate_text(bigram_model, max_length=8)
    print(f"{i+1}. {text}")

---
## Summary

### Named Entity Recognition (NER)

1. **Dictionary-based NER**
   - Simple and fast
   - Requires manual curation
   - Best for known entities

2. **CRF-based NER**
   - Machine learning approach
   - Learns from context
   - Better generalization

### N-gram Language Models

1. **Smoothing**
   - Laplace (Add-one): Simple but over-smooths
   - Add-k: More flexible
   - Good-Turing: Better probability distribution

2. **Backoff**
   - Uses lower-order n-grams when data is sparse
   - Improves robustness

3. **Perplexity**
   - Evaluation metric
   - Lower is better
   - Measures model uncertainty

### Key Takeaways

- NER is crucial for information extraction
- Dictionary-based methods are simple but limited
- CRF-based methods provide better generalization
- N-gram models are fundamental for language modeling
- Smoothing is essential to handle unseen n-grams
- Backoff improves robustness with sparse data
- Perplexity quantifies model quality

---
## Exercises

1. **NER Exercise**: Expand the dictionary-based NER with more entity types (DATE, MONEY, etc.)

2. **CRF Exercise**: Add more features to the CRF model (word prefixes, word length, etc.)

3. **N-gram Exercise**: Implement a trigram model and compare with bigram

4. **Smoothing Exercise**: Implement Good-Turing smoothing

5. **Application Exercise**: Build a text autocomplete system using n-grams